In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [2]:
from modules.data.conll2003.prc import conll2003_preprocess

In [3]:
data_dir = "data/conll2003"

In [4]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [5]:
from modules.data import bert_data

In [6]:
data = bert_data.LearnData.create(
    train_df_path="data/conll2003/eng.train.train.csv",
    valid_df_path="data/conll2003/eng.testa.dev.csv",
    idx2labels_path="data/conll2003/idx2labels2.txt",
    clear_cache=True,
    model_name="bert-base-cased"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [7]:
from modules.models.bert_models import BERTBiLSTMCRF

In [8]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3)

In [9]:
from modules.train.train import NerLearner

In [10]:
num_epochs = 100

In [11]:
learner = NerLearner(
    model, data, "models/conll2003-BERTBiLSTMCRF-base-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [12]:
model.get_n_trainable_params()

2235023

In [ ]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 0.


### Predict

In [11]:
from modules.data.bert_data import get_data_loader_for_predict

In [12]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [13]:
preds = learner.predict(dl)

In [14]:
from sklearn_crfsuite.metrics import flat_classification_report

In [15]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [16]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [17]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [18]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9536    0.9431    0.9483      2005
         I_O     0.9973    0.9969    0.9971     41651
      I_MISC     0.9163    0.9155    0.9159      1255
       I_PER     0.9842    0.9863    0.9853      2848
       I_LOC     0.9674    0.9584    0.9629      1922

   micro avg     0.9916    0.9905    0.9911     49681
   macro avg     0.9638    0.9600    0.9619     49681
weighted avg     0.9916    0.9905    0.9911     49681



### Test

In [28]:
from modules.data.bert_data import get_data_loader_for_predict

In [29]:
dl = get_data_loader_for_predict(data, df_path="/home/eartemov/ae/work/conll2003/eng.testb.dev.csv")

In [30]:
preds = learner.predict(dl)

In [31]:
from sklearn_crfsuite.metrics import flat_classification_report

In [32]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [33]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [34]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [26]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8933    0.9081    0.9006      2360
         I_O     0.9953    0.9921    0.9937     37492
      I_MISC     0.8260    0.8088    0.8173       910
       I_PER     0.9727    0.9810    0.9768      2683
       I_LOC     0.9219    0.9300    0.9259      1815

   micro avg     0.9822    0.9808    0.9815     45260
   macro avg     0.9218    0.9240    0.9229     45260
weighted avg     0.9823    0.9808    0.9815     45260

